In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import wget

In [13]:
web_page_2006 = BeautifulSoup(open('../Top_100_songs/HTMLs/Top_100_2006.html'), 'html.parser')

In [14]:
web_page_2006

<!DOCTYPE html>

<html class="" lang="">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
<title>Hot 100 Songs - Year-End | Billboard</title>
<meta content="Hot 100 Songs - Year-End" name="title" property="title">
<meta content="See Billboard's rankings of this year's most popular songs, albums, and artists." name="description" property="description">
<meta content="https://www.billboard.com/assets/1623447255/images/ye-charts/charts-ye-share-fb.jpg?a21009ae4c6e0a3e1f2c" name="og:image" property="og:image">
<meta content="https://www.billboard.com/assets/1623447255/images/ye-charts/charts-ye-share-twitter.jpg?a21009ae4c6e0a3e1f2c" name="twitter:image" property="twitter:image"/>
<meta content="@billboard" name="twitter:site"/>
<meta content="Billboard" property="og:site_name">
<meta content="article" property="og:type"/>
<script async="async" data-cfasync="false" s

In [37]:
all_titles_2006 = web_page_2006.select('.ye-chart-item__title')
all_artist_2006 = web_page_2006.select('.ye-chart-item__artist')
all_ranks_2006 = web_page_2006.select('.ye-chart-item__rank')

In [40]:
ranks_list = []
for rank in all_ranks_2006:
    ranks_list.append(rank.get_text().replace('\n', ''))

In [30]:
titles_list = []
for title in all_titles_2006:
    titles_list.append(title.get_text().replace('\n', ''))

In [34]:
artist_list = []
for artist in all_artist_2006:
    artist_list.append(artist.get_text().replace('\n', ''))

In [74]:
year_list = []
for year in web_page_2006.select('.ye-chart-item__primary-row'):
    year_list.append(year['data-date'])

In [89]:
df_2006 = pd.DataFrame(data = [ranks_list, titles_list, artist_list, year_list]).T
df_2006.columns = ['Rank', 'Title', 'Artist', 'Year']

In [90]:
df_2006

,Rank,Title,Artist,Year
0,1,Bad Day,Daniel Powter,2006
1,2,Temperature,Sean Paul,2006
2,3,Promiscuous,Nelly Furtado Featuring Timbaland,2006
3,4,You're Beautiful,James Blunt,2006
4,5,Hips Don't Lie,Shakira Featuring Wyclef Jean,2006
...,...,...,...,...
95,96,Stupid Girls,P!nk,2006
96,97,Feel Good Inc,Gorillaz,2006
97,98,I'm Sprung,T-Pain,2006
98,99,Do I Make You Proud,Taylor Hicks,2006


# Juodrastis

In [33]:
# web_page_2006.select('.ye-chart-item__title')[0].get_text().replace('\n', '')